<a href="https://colab.research.google.com/github/Kazuto-Takahashi/Research/blob/main/SEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spikingjelly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 8.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.Functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from spikingjelly.activation_based import layer, neuron

ModuleNotFoundError: No module named 'torch.nn.Functional'

# Model

In [ ]:
class SNN(nn.Module):
    def __init__(self):
        super(SNN, self).__init__()
        self.downsample1 = layer.SeqToANNContainer(
            nn.Conv2d(3, 32, 1, 2, bias=False),
            nn.BatchNorm2d(32)
            )
        self.downsample2 = layer.SeqToANNContainer(
            nn.Conv2d(32, 64, 1, 2, bias=False),
            nn.BatchNorm2d(64)
        )
        self.conv1 = layer.SeqToANNContainer(
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32)
        )
        self.sn1 = neuron.IFNode(detach_reset=True, step_mode='m')
        self.conv2 = layer.SeqToANNContainer(
            nn.Conv2d(32, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(64)
        )
        self.sn2 = neuron.IFNode(detach_reset=True, step_mode='m')
        self.conv3 = layer.SeqToANNContainer(
            nn.Conv2d(32, 64, 3, 2, 1, bias=False),
            nn.BatchNorm2d(64)
        )
        self.sn3 = neuron.IFNode(detach_reset=True, step_mode='m')
        self.conv4 = layer.SeqToANNContainer(
            nn.Conv2d(64, 64, 3, 1, 1, bias=False),
            nn.BatchNorm2d(64)
        )
        self.sn4 = neuron.IFNode(detach_reset=True, step_mode='m')
        self.mp = layer.SeqToANNContainer(
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 10)
        )
        self.sn5 = neuron.IFNode(detach_reset=True, step_mode='m')

    def forward(self, x):
        identity = x
        x = self.sn1(self.conv1(x))
        x = self.conv2(x))
        identity = self.downsample1(identity)
        x += identity
        x = self.sn2(x)
        identity = x
        x = self.sn3(self.conv3(x))
        x = self.conv4(x)
        identity = self.downsample2(identity)
        x += identity
        x = self.sn4(x)
        x = self.mp(x)
        x = self.sn5(x)
        print(x.shape)
        return x

# Training

In [ ]:
model = SNN()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

tf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])
train = datasets.CIFAR10('./data', train=True, download=True, transform=tf)
test = datasets.CIFAR10('./data', train=False, download=True, transform=tf)
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=True)

epochs = 10
for epoch in range(epochs):
    smodel.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()